# Offset

In [4]:
import threading
import time
import numpy as np
import sounddevice as sd
import serial
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import matplotlib.pyplot as plt
from ordered_set import OrderedSet
import pickle

In [5]:
import sys
sys.path.append('..')
# Custom imports
from analysis_utils import DataContainer
from audio_constants import ref_volume, dB_to_scalar

In [6]:
ESP_COM_PORT = "COM22"
ESP_BAUDRATE = 115200

In [7]:
# Get the default audio playback device
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)

# Create a volume control interface
volume_controller = cast(interface, POINTER(IAudioEndpointVolume))

In [28]:
# Function to receive serial data and store it
def receive_serial_data(data):
    ser = serial.Serial(ESP_COM_PORT, ESP_BAUDRATE)  # Replace 'COM1' with your serial port
    while not finish_flag:
        received_data = ser.readline().decode().strip()
        received_data = received_data.split(" ")[0]
        try:
            received_data = float(received_data)
        except ValueError:
            received_data = float(received_data.split(".")[0])
        try:
            if log_flag:
                data = np.append(data, received_data)
                print(received_data)
        except KeyError:
            pass
    ser.close()

In [25]:
# Function to play audio sine wave
def play_test_wave():
    global log_flag, finish_flag
    duration = 15 if TIME_PERIOD == 0.125 else 30 # s
    sample_rate = 88200
    t = np.arange(0, duration, 1 / sample_rate)

    sine_wave = np.sin(2 * np.pi * 1000 * t)
    volume_controller.SetMasterVolumeLevelScalar(dB_to_scalar[ref_volume], None)

    finish_flag = False
    log_flag = True

    # Wait before playing the sine wave
    time.sleep(0.375 if TIME_PERIOD == 0.125 else 3) #s

    sd.play(sine_wave, sample_rate, blocking=True)

    # Wait after playback is done
    time.sleep(0.375 if TIME_PERIOD == 0.125 else 3) #s

    log_flag = False

    # Send a signal to the other thread to terminate
    finish_flag = True

In [8]:
# Create formatted containers to store received serial data
offset_data_F_unwted = np.array([])
offset_data_S_unwted = np.array([])
offset_data_F_wted = np.array([])
offset_data_S_wted = np.array([])

# Flag to signal termination
finish_flag = False
# Flag to signal when to log measurements
log_flag = False

## Ejecutar pruebas

### Periodo F, sin ponderación A

In [ ]:
TIME_PERIOD = 0.125

# Create the serial data receiving thread
serial_thread = threading.Thread(target=receive_serial_data, args=(offset_data_F_unwted,), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_test_wave, daemon=True)

# Start serial thread
serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
serial_thread.join()
audio_thread.join()

# Reset the flag for next test
finish_flag = False

### Periodo S, sin ponderación A

In [ ]:
TIME_PERIOD = 1.0

# Create the serial data receiving thread
serial_thread = threading.Thread(target=receive_serial_data, args=(offset_data_S_unwted,), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_test_wave, daemon=True)

# Start serial thread
serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
serial_thread.join()
audio_thread.join()

# Reset the flag for next test
finish_flag = False

### Periodo F, con ponderación A

In [ ]:
TIME_PERIOD = 0.125

# Create the serial data receiving thread
serial_thread = threading.Thread(target=receive_serial_data, args=(offset_data_F_wted,), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_test_wave, daemon=True)

# Start serial thread
serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
serial_thread.join()
audio_thread.join()

# Reset the flag for next test
finish_flag = False

### Periodo S, con ponderación A

In [ ]:
TIME_PERIOD = 1.0

# Create the serial data receiving thread
serial_thread = threading.Thread(target=receive_serial_data, args=(offset_data_S_wted,), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_test_wave, daemon=True)

# Start serial thread
serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
serial_thread.join()
audio_thread.join()

# Reset the flag for next test
finish_flag = False

## Guardar datos

In [7]:
offset_data_F_unwted = np.array([72, 71.9, 71.9, 71.9, 72, 72, 71.9, 72.2, 72.3, 72.1, 72.2, 72.1, 72, 72.1, 72.2
])
offset_data_S_unwted = np.array([72.2, 72.1, 72.1, 72.1, 72, 72.1, 72, 72, 72, 72.1, 72.2, 72.2, 72.2, 72.1, 72.3, 71.7
])
offset_data_F_wted = np.array([70, 71.3, 71.3, 71.2, 71, 71, 71, 71.9, 71.8, 71.4, 71.7, 72.1])
offset_data_S_wted = np.array([70.2, 72.1, 72, 72.1, 72.3, 72.1, 72.2, 72.1, 72, 72.1, 72, 72.1, 72, 72, 72, 72, 72, 72.1, 72.1, 72.2, 72.2, 72.1, 72.1, 72.2, 72.2
])

In [8]:
offset_data_F_unwted_ref = np.array([70.6, 69.9, 70, 69.2, 69.6, 69.2, 69.3, 69.6, 69.9, 69.7, 69.7, 70, 69.6, 69.6, 66.4
])
offset_data_S_unwted_ref = np.array([70, 70, 70.1, 70.6, 70.3, 70.4, 70.3, 70.4, 70.3, 70.4, 70.3, 70.3, 70.2, 70.4, 70.5, 70.8
])
offset_data_F_wted_ref = np.array([68.7, 69.7, 69.8, 70, 69.6, 69.5, 69.6, 69.4, 69.3, 69.4, 69.4, 69.4])
offset_data_S_wted_ref = np.array([69.2, 69.6, 69.7, 69.9, 69.6, 70, 69.6, 69.7, 69.8, 69.6, 69.7, 69.7, 69.6, 69.7, 69.7, 69.7, 70.6, 69.8, 69.7, 69.8, 69.7, 69.8, 70.3, 69.8, 69.7])

In [9]:
with open("../results/offset/F_unwted.pkl", "wb") as file:
    pickle.dump(offset_data_F_unwted, file)

with open("../results/offset/S_unwted.pkl", "wb") as file:
    pickle.dump(offset_data_S_unwted, file)

with open("../results/offset/F_wted.pkl", "wb") as file:
    pickle.dump(offset_data_F_wted, file)

with open("../results/offset/S_wted.pkl", "wb") as file:
    pickle.dump(offset_data_S_wted, file)

In [ ]:
with open("../results/offset/F_unwted_ref.pkl", "wb") as file:
    pickle.dump(offset_data_F_unwted_ref, file)

with open("../results/offset/S_unwted_ref.pkl", "wb") as file:
    pickle.dump(offset_data_S_unwted_ref, file)

with open("../results/offset/F_wted_ref.pkl", "wb") as file:
    pickle.dump(offset_data_F_wted_ref, file)

with open("../results/offset/S_wted_ref.pkl", "wb") as file:
    pickle.dump(offset_data_S_wted_ref, file)